# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [13]:
DATA_FOLDER = '../../Tutorials/02 - Intro to Pandas/Data/' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

Potential problems:
- Column name (same for rows) + inconsistent case sensitive
Date,Variable,National
date,variable,...National
Date,Description,Totals

- Split rows
12/1/2014,"Total death/s in confirmed, 
 probable, suspected cases",3161,158,124,10,63,74,5,26,384,549,17,1651,53,8,25,14,
 
- Unnamed/partial columns
- Some rows are counts of others
New cases of suspects, New cases of probables, New cases of confirmed, Total new cases registered so far

In [38]:
import glob
import pandas as pd
import os

countries_paths = glob.glob(DATA_FOLDER + 'ebola/*/')
countries_names = [os.path.basename(os.path.normpath(x)).replace('_data', '').capitalize() for x in countries]

In [40]:
print(countries_names)

['Guinea', 'Liberia', 'Sl']


In [76]:
aliases = {
    'variable': 'description',
    'national': 'totals'
}

In [281]:
dfs = []
for i, path in enumerate(countries_paths):
    filenames = glob.glob(path + "/*.csv")
    
    for f in filenames:
        df = pd.read_csv(f)
        df.columns = [c.lower() for c in df.columns] # Set all columns to lower-case
        df.rename(columns=aliases, inplace=True)
        df['country'] = countries_names[i]
        df['date'] = [parser.parse(x, tzinfos={'dayfirst': True}) for x in df['date']]
        df['totals'] = pd.to_numeric(df['totals'], errors='coerce')
        df = df.fillna(0)
        dfs.append(df[['date', 'country', 'description', 'totals']])
        
df = pd.concat(dfs)
df['description'] = [x.lower() for x in df['description']]
#print(df['description'].unique())

In [282]:
from itertools import product

column_keywords = {
    'new_cases': ['new cases of suspects', 'new cases of probables', 'new cases of confirmed', 'new case/s (suspected)',
                 'new case/s (probable)', 'new case/s (confirmed)',  'new_suspected', 'new_probable', 'new_confirmed'],
    'new_deaths': ['new deaths registered today', 'new deaths registered', 'newly reported deaths',
                  'etc_new_deaths']
}

desc_map = []
for k, v in column_keywords.items():
    for alias in v:
        desc_map.append((k, alias))
desc_map = pd.DataFrame(desc_map, columns=['type', 'description'])
df = df.merge(desc_map, on='description', how='inner')
df = df.drop('description', axis=1)

In [283]:
df.head()

,date,country,totals,type
0,2014-08-04,Guinea,5.0,new_cases
1,2014-08-26,Guinea,18.0,new_cases
2,2014-08-27,Guinea,12.0,new_cases
3,2014-08-30,Guinea,15.0,new_cases
4,2014-08-31,Guinea,9.0,new_cases


In [284]:
pivoted = df.pivot_table(index=['date', 'country'], columns='type', values='totals', aggfunc='sum')
#pivoted = pivoted['totals'].reset_index()
pivoted.columns.name = None
df = pivoted.reset_index()

In [243]:
from dateutil import parser
df_ = df.groupby([df.date, df.country, df.type]).sum().reset_index()
df_.head()

,date,country,type,totals
0,2014-06-16,Liberia,new_cases,4.0
1,2014-06-16,Liberia,new_deaths,2.0
2,2014-06-17,Liberia,new_cases,2.0
3,2014-06-17,Liberia,new_deaths,0.0
4,2014-06-22,Liberia,new_cases,10.0


In [285]:
df[df.country == 'Liberia'].head()

,date,country,new_cases,new_deaths
0,2014-06-16,Liberia,4.0,2.0
1,2014-06-17,Liberia,2.0,0.0
2,2014-06-22,Liberia,10.0,4.0
3,2014-06-24,Liberia,6.0,4.0
4,2014-06-25,Liberia,7.0,3.0


In [297]:
df_ = df.groupby([df.country, df.date.dt.year, df.date.dt.month]).mean()
df_.index.names = ['country', 'year', 'month']
df_.reset_index().sort_values(ascending=[True, True, True], by=['country', 'year', 'month'])

,country,year,month,new_cases,new_deaths
0,Guinea,2014,8,25.800000,3.400000
1,Guinea,2014,9,19.625000,3.562500
2,Guinea,2014,10,34.000000,15.000000
3,Liberia,2014,6,5.714286,2.000000
4,Liberia,2014,7,8.545455,4.272727
5,Liberia,2014,8,37.222222,23.222222
6,Liberia,2014,9,63.833333,36.041667
7,Liberia,2014,10,45.560000,28.040000
8,Liberia,2014,11,26.466667,13.466667
9,Liberia,2014,12,5178.555556,0.000000


In [58]:
pd.read_csv('C:\\Users\\Dario\\Documents\\Università\\ADA\\Tutorials\\02 - Intro to Pandas\\Data\\ebola\\liberia_data\\2014-12-01.csv')

,Date,Variable,National,Bomi County,Bong County,Gbarpolu County,Grand Bassa,Grand Cape Mount,Grand Gedeh,Grand Kru,Lofa County,Margibi County,Maryland County,Montserrado County,Nimba County,River Gee County,RiverCess County,Sinoe County,Unnamed: 18
0,12/1/2014,Specimens collected,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12/1/2014,Specimens pending for testing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12/1/2014,Total specimens tested,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12/1/2014,Newly reported deaths,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12/1/2014,Total death/s in confirmed cases,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,12/1/2014,Total death/s in probable cases,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,12/1/2014,Total death/s in suspected cases,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,12/1/2014,"Total death/s in confirmed, \r\n probable, sus...",3161.0,158.0,124.0,10.0,63.0,74.0,5.0,26.0,384.0,549.0,17.0,1651.0,53.0,8.0,25.0,14.0,NaN
8,12/1/2014,Case Fatality Rate (CFR) - \r\n Confirmed & Pr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,12/1/2014,Newly reported contacts,263.0,0.0,16.0,0.0,11.0,19.0,0.0,0.0,0.0,34.0,0.0,181.0,0.0,0.0,0.0,2.0,NaN


## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [ ]:
# Write your answer here

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here